In [1]:
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [2]:
fname = 'personal_finance.xlsx'

accounts = pd.read_excel(fname, sheet_name='accounts')
incomes = pd.read_excel(fname, sheet_name='incomes')
expenses = pd.read_excel(fname, sheet_name='expenses')


In [3]:
expenses.head()

,date,account,description,currency,amount,category
0,2019-12-31,sella,balance,EUR,142.33,wants
1,2019-12-31,sella,savings,EUR,12000.00,savings
2,2020-01-31,sella,balance,EUR,131.30,wants
3,2020-04-30,sella,balance,EUR,0.98,wants
4,2020-06-30,sella,balance,EUR,1056.00,wants


In [4]:
expenses['amount'] = expenses.amount * -1

In [5]:
expenses.head()

,date,account,description,currency,amount,category
0,2019-12-31,sella,balance,EUR,-142.33,wants
1,2019-12-31,sella,savings,EUR,-12000.00,savings
2,2020-01-31,sella,balance,EUR,-131.30,wants
3,2020-04-30,sella,balance,EUR,-0.98,wants
4,2020-06-30,sella,balance,EUR,-1056.00,wants


In [6]:
df = pd.concat([incomes,expenses])

In [7]:
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year

In [8]:
grouped = df.groupby(['year', 'month', 'account', 'category', 'currency']).agg({'amount':'sum'}).reset_index()

In [9]:
sella = grouped[grouped.account == 'sella']
sella['balance'] = sella['amount'].cumsum()
sella['account'] = 'sella'
sella

,year,month,account,category,currency,amount,balance
0,2019,11,sella,income,EUR,17606.36,17606.360000
1,2019,12,sella,income,EUR,921.00,18527.360000
2,2019,12,sella,savings,EUR,-12000.00,6527.360000
3,2019,12,sella,wants,EUR,-142.33,6385.030000
4,2020,1,sella,income,EUR,1918.00,8303.030000
...,...,...,...,...,...,...,...
137,2024,3,sella,wants,EUR,-675.64,5620.380273
139,2024,4,sella,needs,EUR,-319.52,5300.860273
140,2024,4,sella,rent,EUR,1212.00,6512.860273
141,2024,4,sella,transfer,EUR,-500.00,6012.860273


In [10]:
# sella.groupby(['year', 'month']).agg({'amount':'sum', 'balance':'last'})

needs = pd.pivot(
    data=sella.drop(['account', 'currency', 'balance'], axis=1),
    index=['year', 'month'],
    columns='category'
).fillna(0.0)[('amount','needs')]

wants = pd.pivot(
    data=sella.drop(['account', 'currency', 'balance'], axis=1),
    index=['year', 'month'],
    columns='category'
).fillna(0.0)[('amount','wants')]

savings = pd.pivot(
    data=sella.drop(['account', 'currency', 'balance'], axis=1),
    index=['year', 'month'],
    columns='category'
).fillna(0.0)[('amount','savings')]

# pd.merge(
#     left=sella,
#     right=needs.reset_index(),
#     on=['year','month'],
#     # right_on=[('year', ''), ('month', '')],
#     how='left'
# )

pd.concat([sella, needs.reset_index()], keys=['year', 'month'])

year  month account category currency    amount   balance  \
year  0   2019.0   11.0   sella   income      EUR  17606.36  17606.36   
      1   2019.0   12.0   sella   income      EUR    921.00  18527.36   
      2   2019.0   12.0   sella  savings      EUR -12000.00   6527.36   
      3   2019.0   12.0   sella    wants      EUR   -142.33   6385.03   
      4   2020.0    1.0   sella   income      EUR   1918.00   8303.03   
...          ...    ...     ...      ...      ...       ...       ...   
month 49     NaN    NaN     NaN      NaN      NaN       NaN       NaN   
      50     NaN    NaN     NaN      NaN      NaN       NaN       NaN   
      51     NaN    NaN     NaN      NaN      NaN       NaN       NaN   
      52     NaN    NaN     NaN      NaN      NaN       NaN       NaN   
      53     NaN    NaN     NaN      NaN      NaN       NaN       NaN   

          (year, )  (month, )  (amount, needs)  
year  0        NaN        NaN              NaN  
      1        NaN        NaN              NaN  
      2        NaN        NaN              NaN  
      3        NaN        NaN              NaN  
      4        NaN        NaN              NaN  
...            ...        ...              ...  
month 49    2023.0       12.0             0.00  
      50    2024.0        1.0             0.00  
      51    2024.0        2.0         -4848.00  
      52    2024.0        3.0         -2883.61  
      53    2024.0        4.0          -319.52  

[162 rows x 10 columns]

In [11]:
sella.groupby(['year','month']).agg({'amount':'sum', 'balance':'last'})

amount       balance
year month                            
2019 11     17606.360000  17606.360000
     12    -11221.330000   6385.030000
2020 1       1786.700000   8171.730000
     2       1595.000000   9766.730000
     3       1584.000000  11350.730000
     4       1582.020000  12932.750000
     5       1578.000000  14510.750000
     6        513.000000  15023.750000
     7       1123.500000  16147.250000
     8      -3788.320000  12358.930000
     9      -1346.570000  11012.360000
     10      3771.070000  14783.430000
     11       614.000000  15397.430000
     12       545.180000  15942.610000
2021 1         -1.700000  15940.910000
     2       2447.450000  18388.360000
     3       1706.000000  20094.360000
     4        494.610000  20588.970000
     5         18.550000  20607.520000
     6      -5777.570000  14829.950000
     7       -440.650000  14389.300000
     8       1196.040000  15585.340000
     9       2748.750000  18334.090000
     10      -585.850000  17748.240000
     11     -1216.370000  16531.870000
     12    -12276.700000   4255.170000
2022 1       1096.340000   5351.510000
     2       1563.580000   6915.090000
     3       -140.780000   6774.310000
     4       1266.030000   8040.340000
     5       1718.180000   9758.520000
     6      19250.030000  29008.550000
     7       5585.950000  34594.500000
     8       3467.440000  38061.940000
     9       3119.020000  41180.960000
     10      1125.250000  42306.210000
     11      1931.090000  44237.300000
     12    -35289.500000   8947.800000
2023 1       1396.730000  10344.530000
     2       2311.850000  12656.380000
     3        450.980000  13107.360000
     4       1070.390000  14177.750000
     5      -6707.020000   7470.730000
     6       1206.270000   8677.000000
     7        997.270000   9674.270000
     8      -1510.700000   8163.570000
     9       -170.230000   7993.340000
     10      -373.960000   7619.380000
     11      -475.410000   7143.970000
     12       -92.499727   7051.470273
2024 1        175.070000   7226.540273
     2      -4619.020000   2607.520273
     3       3012.860000   5620.380273
     4        367.380000   5987.760273

In [12]:
sella.info()
sella[sella.year == 2023]

<class 'pandas.core.frame.DataFrame'>
Index: 108 entries, 0 to 142
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      108 non-null    int32  
 1   month     108 non-null    int32  
 2   account   108 non-null    object 
 3   category  108 non-null    object 
 4   currency  108 non-null    object 
 5   amount    108 non-null    float64
 6   balance   108 non-null    float64
dtypes: float64(2), int32(2), object(3)
memory usage: 5.9+ KB


,year,month,account,category,currency,amount,balance
87,2023,1,sella,income,EUR,1978.000000,10925.800000
88,2023,1,sella,wants,EUR,-581.270000,10344.530000
89,2023,2,sella,income,EUR,2397.000000,12741.530000
90,2023,2,sella,wants,EUR,-85.150000,12656.380000
91,2023,3,sella,income,EUR,2237.000000,14893.380000
92,2023,3,sella,wants,EUR,-1786.020000,13107.360000
94,2023,4,sella,income,EUR,2350.000000,15457.360000
95,2023,4,sella,wants,EUR,-1279.610000,14177.750000
96,2023,5,sella,income,EUR,2249.000000,16426.750000
97,2023,5,sella,wants,EUR,-8956.020000,7470.730000


In [13]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
year = 2024
tmp = sella.groupby(['year','month']).agg({'amount':'sum', 'balance':'last'}).reset_index()

fig = px.bar(
    data_frame=tmp[tmp.year == year],
    x='month',
    y='balance'
)

fig.show()